In [1]:
from __future__ import unicode_literals
import os
import nltk
import pandas as pd
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
import numpy as np
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
np.random.seed(1337)  # For Reproducibility
import multiprocessing

Using Theano backend.


In [2]:
# reload necessary data set to pandas DataFrame.
train_df = pd.read_csv('dumps/stemmered_train_mrd.csv', encoding='utf8')
test_df = pd.read_csv('dumps/stemmered_test_mrd.csv', encoding='utf8')

In [3]:
print "Storing terms from training documents as list of lists"
terms_by_doc_train = [document.rstrip(']"').lstrip('"[').split(", ") for document in train_df.ix[:,0]]
terms_by_label_train = train_df.ix[:, 1]
n_terms_per_doc = [len(terms) for terms in terms_by_doc_train]
print "min, max and average number of terms per document:", min(n_terms_per_doc), max(n_terms_per_doc), sum(n_terms_per_doc)/len(n_terms_per_doc)
# print terms_by_doc_train[0][8]
print "Storing terms from test documents as list of lists"
terms_by_doc_test = [document.rstrip(']"').lstrip('"[').split(", ") for document in test_df.ix[:,0]]
terms_by_label_test = test_df.ix[:, 1]
n_terms_per_doc = [len(terms) for terms in terms_by_doc_test]
print "min, max and average number of terms per document:", min(n_terms_per_doc), max(n_terms_per_doc), sum(n_terms_per_doc)/len(n_terms_per_doc)
# print terms_by_doc_test[0][0]
# Store all terms in list
all_terms = [terms for sublist in terms_by_doc_train for terms in sublist]
# Compute average number of terms
avg_len = sum(n_terms_per_doc)/len(n_terms_per_doc)
print "the average number of terms:", avg_len
# Find unique terms
all_unique_terms = list(set(all_terms))
print "the number of unique terms:", len(all_unique_terms)
print "How many sentences in traing set:" , len(terms_by_doc_train)
print "How many sentences in test set:", len(terms_by_doc_test)


Storing terms from training documents as list of lists
min, max and average number of terms per document: 5 39 11
Storing terms from test documents as list of lists
min, max and average number of terms per document: 5 32 11
the average number of terms: 11
the number of unique terms: 11403
How many sentences in traing set: 5794
How many sentences in test set: 3873


In [33]:
# set parameters:
vocab_dim = 20
maxlen = 100
n_iterations = 1  # ideally more..
# Words that appear only once or twice in a billion-word corpus are probably uninteresting typos and garbage. 
n_exposures = 3
window_size = 5
batch_size = 32
n_epoch = 2
input_length = 100
cpu_count = multiprocessing.cpu_count()

In [34]:
combine_train_test_X = terms_by_doc_train + terms_by_doc_train


In [35]:
print('Training a Word2vec model...')
model = Word2Vec(size=vocab_dim,
                 min_count=n_exposures,
                 window=window_size,
                 workers=cpu_count,
                 iter=n_iterations)
model.build_vocab(combine_train_test_X)
model.train(combine_train_test_X)
# print len(model.vocab.viewkeys()) -> 11403

117475

Training a Word2vec model...


In [36]:
gensim_dict = Dictionary()
gensim_dict.doc2bow(model.vocab.keys(), allow_update=True)
# gensim_dict.items() returns [(0, u"'surpris"), (1, u'woodi'), (2, u'yellow'),...]
# K+1 aims at avoiding 0 as index.
w2indx = {v: k+1 for k, v in gensim_dict.items()}
w2vec = {word: model[word] for word in w2indx.keys()}
# print len(model["surpris"]) -> 100
print('Setting up Arrays for Keras Embedding Layer...')
n_symbols = len(w2indx) + 1  # adding 1 to account for 0th index
embedding_weights = np.zeros((n_symbols + 1, vocab_dim))
for word, index in w2indx.items():
    embedding_weights[index, :] = w2vec[word]
# print embedding_weights.shape -> (11405, 100)

Setting up Arrays for Keras Embedding Layer...


In [37]:
X_train = []
for doc in terms_by_doc_train:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_train.append(new_txt)
X_test = []
for doc in terms_by_doc_test:
    new_txt = []
    for word in doc:
        try:
            new_txt.append(w2indx[word])
        except:
            new_txt.append(0)
    X_test.append(new_txt)

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(terms_by_label_train)
y_test = np.array(terms_by_label_test)
print('X_train shape:',  y_train.shape)
print('X_test shape:', y_test.shape)


Pad sequences (samples x time)
(u'X_train shape:', (5794, 100))
(u'X_test shape:', (3873, 100))
(u'X_train shape:', (5794,))
(u'X_test shape:', (3873,))


In [38]:
print('Defining a Simple Keras Model...')
model = Sequential()  # or Graph or whatever
model.add(Embedding(input_dim=n_symbols + 1,
                    output_dim=vocab_dim,
                    mask_zero=True,
                    weights=[embedding_weights],
                    input_length=input_length))  # Adding Input Length
model.add(LSTM(vocab_dim))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
#model.add(Dense(1, activation='relu'))

print('Compiling the Model...')
model.compile(loss='binary_crossentropy', optimizer='adam', class_mode='binary', metrics=['accuracy'])

print("Train...")
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=n_epoch,
          validation_data=(X_test, y_test))

print("Evaluate...")
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

(u'Test score:', 0.55627031720118592)
(u'Test accuracy:', 0.72579395817195969)


3872/3873 [============================>.] - ETA: 0s

3808/3873 [============================>.] - ETA: 0s

3744/3873 [============================>.] - ETA: 0s

3680/3873 [===========================>..] - ETA: 0s

...

 128/5794 [..............................] - ETA: 5s - loss: 0.6954 - acc: 0.4922

  96/5794 [..............................] - ETA: 5s - loss: 0.6951 - acc: 0.5208

  64/5794 [..............................] - ETA: 5s - loss: 0.6969 - acc: 0.5156

Train on 5794 samples, validate on 3873 samples
Epoch 1/2
  32/5794 [..............................] - ETA: 5s - loss: 0.6983 - acc: 0.4688

Defining a Simple Keras Model...
Compiling the Model...
Train...
